In [1]:
import pandas as pd
import re
import folium
from folium import plugins
from folium.plugins import HeatMap
import sqlite3

In [2]:
f= open("result.txt")

In [3]:
lines = f.readlines()

In [4]:
#"2022-05-13 00:08:23.071048 {'lat': 22.6492416, 'lon': 120.307712}
locs = [line for line in lines if re.match("^\d+-\d+-\d.*",line)]

In [5]:
df = pd.DataFrame(locs, columns=['S'])

In [6]:
df.S.str.extract("^(\d+-\d+-\d.*)\s+{")[0]

0      2022-05-13 00:08:23.071048
1      2022-05-13 00:08:26.319361
2      2022-05-13 00:08:29.083294
3      2022-05-13 00:08:32.546873
4      2022-05-13 00:12:59.715069
                  ...            
143    2022-05-21 12:09:14.629422
144    2022-05-21 12:39:59.795315
145    2022-05-21 12:59:05.363073
146    2022-05-21 13:18:03.285418
147    2022-05-21 13:35:38.488062
Name: 0, Length: 148, dtype: object

In [7]:
df['Time'] = pd.to_datetime(df.S.str.extract("^(\d+-\d+-\d.*)\s+{")[0])

In [8]:
df = pd.concat([df.S.str.extract("{'lat': (?P<Lat>\d+\.\d+), 'lon': (?P<Lon>\d+\.\d+)}"),df], axis=1)

In [9]:
df

,Lat,Lon,S,Time
0,22.6492416,120.307712,"2022-05-13 00:08:23.071048 {'lat': 22.6492416,...",2022-05-13 00:08:23.071048
1,22.6492416,120.307712,"2022-05-13 00:08:26.319361 {'lat': 22.6492416,...",2022-05-13 00:08:26.319361
2,22.6492416,120.307712,"2022-05-13 00:08:29.083294 {'lat': 22.6492416,...",2022-05-13 00:08:29.083294
3,22.6492416,120.307712,"2022-05-13 00:08:32.546873 {'lat': 22.6492416,...",2022-05-13 00:08:32.546873
4,22.6492416,120.307712,"2022-05-13 00:12:59.715069 {'lat': 22.6492416,...",2022-05-13 00:12:59.715069
...,...,...,...,...
143,22.6578902,120.30563,"2022-05-21 12:09:14.629422 {'lat': 22.6578902,...",2022-05-21 12:09:14.629422
144,22.6578902,120.30563,"2022-05-21 12:39:59.795315 {'lat': 22.6578902,...",2022-05-21 12:39:59.795315
145,22.6578902,120.30563,"2022-05-21 12:59:05.363073 {'lat': 22.6578902,...",2022-05-21 12:59:05.363073
146,22.6578902,120.30563,"2022-05-21 13:18:03.285418 {'lat': 22.6578902,...",2022-05-21 13:18:03.285418


In [10]:
HMT = []
index_string = []
for i,g in df.set_index("Time").resample("1H"):
    index_string.append(f"{i:%Y/%m/%d, %H:%M}")
    #print(i, [(row['Lat'],row['Lon']) for idx, row in g.iterrows()])
    HMT.append([[row['Lat'],row['Lon']] for idx, row in g.iterrows()])

In [11]:
index_string

['2022/05/13, 00:00',
 '2022/05/13, 01:00',
 '2022/05/13, 02:00',
 '2022/05/13, 03:00',
 '2022/05/13, 04:00',
 '2022/05/13, 05:00',
 '2022/05/13, 06:00',
 '2022/05/13, 07:00',
 '2022/05/13, 08:00',
 '2022/05/13, 09:00',
 '2022/05/13, 10:00',
 '2022/05/13, 11:00',
 '2022/05/13, 12:00',
 '2022/05/13, 13:00',
 '2022/05/13, 14:00',
 '2022/05/13, 15:00',
 '2022/05/13, 16:00',
 '2022/05/13, 17:00',
 '2022/05/13, 18:00',
 '2022/05/13, 19:00',
 '2022/05/13, 20:00',
 '2022/05/13, 21:00',
 '2022/05/13, 22:00',
 '2022/05/13, 23:00',
 '2022/05/14, 00:00',
 '2022/05/14, 01:00',
 '2022/05/14, 02:00',
 '2022/05/14, 03:00',
 '2022/05/14, 04:00',
 '2022/05/14, 05:00',
 '2022/05/14, 06:00',
 '2022/05/14, 07:00',
 '2022/05/14, 08:00',
 '2022/05/14, 09:00',
 '2022/05/14, 10:00',
 '2022/05/14, 11:00',
 '2022/05/14, 12:00',
 '2022/05/14, 13:00',
 '2022/05/14, 14:00',
 '2022/05/14, 15:00',
 '2022/05/14, 16:00',
 '2022/05/14, 17:00',
 '2022/05/14, 18:00',
 '2022/05/14, 19:00',
 '2022/05/14, 20:00',
 '2022/05/

In [12]:
m = folium.Map(location=[df.Lat.iloc[0],df.Lon.iloc[0]], zoom_start=11, )
plugins.HeatMapWithTime(HMT, radius=20,index=index_string, use_local_extrema=True ).add_to(m)

In [13]:
m

In [16]:
HM = []
for i,row in df.iterrows():
    
    #print(i, [(row['Lat'],row['Lon']) for idx, row in g.iterrows()])
    HM.append([row['Lat'],row['Lon']])

In [15]:
df

,Lat,Lon,S,Time
0,22.6492416,120.307712,"2022-05-13 00:08:23.071048 {'lat': 22.6492416,...",2022-05-13 00:08:23.071048
1,22.6492416,120.307712,"2022-05-13 00:08:26.319361 {'lat': 22.6492416,...",2022-05-13 00:08:26.319361
2,22.6492416,120.307712,"2022-05-13 00:08:29.083294 {'lat': 22.6492416,...",2022-05-13 00:08:29.083294
3,22.6492416,120.307712,"2022-05-13 00:08:32.546873 {'lat': 22.6492416,...",2022-05-13 00:08:32.546873
4,22.6492416,120.307712,"2022-05-13 00:12:59.715069 {'lat': 22.6492416,...",2022-05-13 00:12:59.715069
...,...,...,...,...
143,22.6578902,120.30563,"2022-05-21 12:09:14.629422 {'lat': 22.6578902,...",2022-05-21 12:09:14.629422
144,22.6578902,120.30563,"2022-05-21 12:39:59.795315 {'lat': 22.6578902,...",2022-05-21 12:39:59.795315
145,22.6578902,120.30563,"2022-05-21 12:59:05.363073 {'lat': 22.6578902,...",2022-05-21 12:59:05.363073
146,22.6578902,120.30563,"2022-05-21 13:18:03.285418 {'lat': 22.6578902,...",2022-05-21 13:18:03.285418


In [17]:
m = folium.Map(location=[df.Lat.iloc[0],df.Lon.iloc[0]], zoom_start=11, )
plugins.HeatMap(HM, radius=20,index=index_string, use_local_extrema=False ).add_to(m)

In [18]:
m